One obvious fact is that pawpularity could (partly) be determined by pet breed rather than artistic or technical qualities in the images.

Since breed is not part of the metadata I decided to make a quick-and-dirty classification myself. One issue with this is that a lot of these animals are likely not to be purebreeds but still if they have the general caracteristics of a specific breed then that may be enough to influence the pawpularity in a certain direction.

I trained a NN (transfer learning in mobilenet in TensorFlow). The training data came from the the huge "Cat Breeds Dataset" (https://www.kaggle.com/ma7555/cat-breeds-dataset) and the awesome "70 Dog Breeds-Image Data Set" (https://www.kaggle.com/gpiosenka/70-dog-breedsimage-data-set) dataset. The "Cat Breeds Dataset" was unbalanced and noisy which probably influenced quality of my model.

As you will discover below the model is not very precise at this point but still it should give us some idea wether the hypothesis holds or not.

First... let's import the libraries

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf

Then import the data files. For now these are in a private dataset while I work on making a decent model, then later I will release the files as well as the model.

Feel free to use them if you want to. :-)

In [ ]:
model = tf.keras.models.load_model('../input/pet-breed-model-tensorflow-2-keras/petbreed_model/')

pawbreed_df = pd.read_csv('../input/pawpularity-breed-list/breed_list.csv')
pawbreed_label_df = labels_df = pd.read_csv('../input/pawpularity-breed-list/petbreed_labels.csv')
pawpularity_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')

First let us see if the classification can distinguish cats from dogs.

5 images classified as cats by my model. 3 correct and two dogs. Hmm...

In [ ]:
cat_ids = []

for counter in range(10,15):
    cat_id = pawbreed_label_df[pawbreed_label_df['cat_or_dog'] == 1]['id'].values[counter]
    print('Cat breed number ' + str(counter) + ' has class ' + str(cat_id))

    cat_ids += [pawbreed_df[pawbreed_df['Breed'] == cat_id]['Id'].values[0]]

print('First 5 images ids of cats: ' + str(cat_ids))

for ids in cat_ids:
    t1_img = Image.open('../input/petfinder-pawpularity-score/train/' + ids + '.jpg').resize((400, 400), Image.BICUBIC)
    plt.imshow(np.array(t1_img))
    plt.show()

Ok, let us look for dogs then. That's better. All dogs.

In [ ]:
dog_ids = []

for counter in range(10,15):
    dog_id = pawbreed_label_df[pawbreed_label_df['cat_or_dog'] == 0]['id'].values[counter]
    print('Dog breed number ' + str(counter) + ' has class ' + str(dog_id))

    dog_ids += [pawbreed_df[pawbreed_df['Breed'] == dog_id]['Id'].values[0]]

print('First 5 images ids of dogs: ' + str(dog_ids))

for ids in dog_ids:
    t1_img = Image.open('../input/petfinder-pawpularity-score/train/' + ids + '.jpg').resize((400, 400), Image.BICUBIC)
    plt.imshow(np.array(t1_img))
    plt.show()

Being more a dog person than a cat person all cats look more or less the same to me. Persian seem to stand out however with it's broad, furry head. All 5 look like Persian to me.

In [ ]:
persian_breed_id = pawbreed_label_df[pawbreed_label_df['label'] == 'Persian']['id'].values[0]
print('Siamese breed id is ' + str(persian_breed_id))

persian_ids = pawbreed_df[pawbreed_df['Breed'] == persian_breed_id]['Id'].values[:5]

print('First 5 images ids of this breed: ' + str(persian_ids))

for ids in persian_ids:
    t1_img = Image.open('../input/petfinder-pawpularity-score/train/' + ids + '.jpg').resize((400, 400), Image.BICUBIC)
    plt.imshow(np.array(t1_img))
    plt.show()

Ok... that seemed to work well. How about a dog next? Poodle images?

Yep. Looks ok as well.

In [ ]:
poodle_breed_id = pawbreed_label_df[pawbreed_label_df['label'] == 'Poodle']['id'].values[0]
print('Poodle breed id is ' + str(poodle_breed_id))

poodle_ids = pawbreed_df[pawbreed_df['Breed'] == poodle_breed_id]['Id'].values[:5]

print('First 5 images ids of this breed: ' + str(poodle_ids))

for ids in poodle_ids:
    t1_img = Image.open('../input/petfinder-pawpularity-score/train/' + ids + '.jpg').resize((400, 400), Image.BICUBIC)
    plt.imshow(np.array(t1_img))
    plt.show()

Let's have a quick look at the model itself.

Input image size (since it's an Inception model) is 299 by 299 pixels.

Let's put the two falsely classified images through it (the dogs that the model called cats). What we see that the model is not very confident on the first image. It basically tells us that it could be anything. :-)

The second image the model is fairly confident that it is a hairless cat. Poor dog...

In [ ]:
model.summary()

image_size = 299

img = Image.open('../input/petfinder-pawpularity-score/train/063c62c2be5f2f7abfc5e2ff5ba014ab.jpg').resize((image_size, image_size))
plt.imshow(np.array(img))
plt.show()

res = model.predict(np.asarray(img).reshape(1, image_size, image_size, 3))

best_scores = np.argsort(res*-1)[0,:3]

for i in range(0, 3):
    print('Confidence: ' + str(res[0,best_scores[i]]))
    print('Breed: ' + labels_df.iloc[best_scores[i]]['label'])
    
img = Image.open('../input/petfinder-pawpularity-score/train/06738f64265a7371bd0788fbfd95b510.jpg').resize((image_size, image_size))
plt.imshow(np.array(img))
plt.show()

res = model.predict(np.asarray(img).reshape(1, image_size, image_size, 3))

best_scores = np.argsort(res*-1)[0,:3]

for i in range(0, 3):
    print('Confidence: ' + str(res[0,best_scores[i]]))
    print('Breed: ' + labels_df.iloc[best_scores[i]]['label'])

Let's try something that worked better. First one of the Persian cats and then one of the Poodles. 

Looks fine.

In [ ]:
img = Image.open('../input/petfinder-pawpularity-score/train/1a10978f92e91f0a14ede103fbc09e29.jpg').resize((image_size, image_size))
plt.imshow(np.array(img))
plt.show()

res = model.predict(np.asarray(img).reshape(1, image_size, image_size, 3))

best_scores = np.argsort(res*-1)[0,:3]

for i in range(0, 3):
    print('Confidence: ' + str(res[0,best_scores[i]]))
    print('Breed: ' + labels_df.iloc[best_scores[i]]['label'])
    
img = Image.open('../input/petfinder-pawpularity-score/train/04fef9f129bc6e4b90644d4290fde8c3.jpg').resize((image_size, image_size))
plt.imshow(np.array(img))
plt.show()

res = model.predict(np.asarray(img).reshape(1, image_size, image_size, 3))

best_scores = np.argsort(res*-1)[0,:3]

for i in range(0, 3):
    print('Confidence: ' + str(res[0,best_scores[i]]))
    print('Breed: ' + labels_df.iloc[best_scores[i]]['label'])

What's next? Let's take a look at the amount of images in each class/breed.

It seems that there are quite a few classes with only a single image, and a few very big ones with more than 2000 images in the largest class.

This could be because the breeds are very popular or because the model is wrong (which is likely since the model finds it difficult to discern between the different cat breeds). It is likely that class 116 and 108 is "generic cat" rather than "Siamese" and "Tabby". This would warrant further investigation.

In [ ]:
counts = pawbreed_df.groupby('Breed')['Pawpularity'].count()

print('5 smallest groups:')
print(counts.nsmallest(5))

print('5 largest groups:')
print(counts.nlargest(5))

fig1, ax1 = plt.subplots()
ax1.pie(counts, shadow=True, startangle=90)
ax1.axis('equal')

plt.show()

Ok, let's have a look at the median pawpularity for each breed.

Well... a street dog, a tail-less cat, half the head of a cat drinking milk seem to be some of the least pawpular pets.

It seems fluffy cats and dogs are most pawpular...

In [ ]:
median = pawbreed_df.groupby('Breed')['Pawpularity'].median()

print('3 least pawpular classes by median:')
least = median.nsmallest(3)
print(least)

for index, row in least.iteritems():
    print(index)
    print('Pawularity: ' + str(row))
    
    median_ids = pawbreed_df[pawbreed_df['Breed'] == index]['Id'].values[:3]

    for ids in median_ids:
        t1_img = Image.open('../input/petfinder-pawpularity-score/train/' + ids + '.jpg').resize((400, 400), Image.BICUBIC)
        plt.imshow(np.array(t1_img))
        plt.show()
        
print('3 most popular classes by median:')
most = median.nlargest(3)
print(most)

for index, row in most.iteritems():
    print(index)
    print('Pawularity: ' + str(row))
    
    median_ids = pawbreed_df[pawbreed_df['Breed'] == index]['Id'].values[:3]

    for ids in median_ids:
        t1_img = Image.open('../input/petfinder-pawpularity-score/train/' + ids + '.jpg').resize((400, 400), Image.BICUBIC)
        plt.imshow(np.array(t1_img))
        plt.show()


Alright then... let's try average pawpularity (knowing that it will at least partly give us the same results).

The least pawpular are still the same, for the same reasons (small groups and technical qualities of the images probably weighs more than breed here).

For the most popular ones we now have 82, Nebelung as number 3, which also is a fluffy breed and 78, Maltese again as number 2.

I feel there is a pattern emerging now even though it is not very clear. Maybe someone should make a fluffy-classifier instead. :-)

In [ ]:
average = pawbreed_df.groupby('Breed')['Pawpularity'].mean()

print('3 least pawpular classes by average:')
least = average.nsmallest(3)
print(least)

for index, row in least.iteritems():
    print(index)
    print('Pawularity: ' + str(row))
    
    average_ids = pawbreed_df[pawbreed_df['Breed'] == index]['Id'].values[:3]

    for ids in average_ids:
        t1_img = Image.open('../input/petfinder-pawpularity-score/train/' + ids + '.jpg').resize((400, 400), Image.BICUBIC)
        plt.imshow(np.array(t1_img))
        plt.show()
        
print('3 most popular classes by average:')
most = average.nlargest(3)
print(most)

for index, row in most.iteritems():
    print(index)
    print('Pawularity: ' + str(row))
    
    median_ids = pawbreed_df[pawbreed_df['Breed'] == index]['Id'].values[:3]

    for ids in median_ids:
        t1_img = Image.open('../input/petfinder-pawpularity-score/train/' + ids + '.jpg').resize((400, 400), Image.BICUBIC)
        plt.imshow(np.array(t1_img))
        plt.show()


I really _do_ think fluffy pets have an unfair advantage to less hairy ones. Let's have a look and see if I am correct.

First, let's have a look at Sphynx, a hairless cat breed, and American Hairless and compare them to Domestic Long Hair and Maltese (which we already know is popular).

It could be a coincidence but it seems like there is something here...

In [ ]:
print('Sphynx average Pawpularity: ' + str(average[115]))
print('American Hairless average Pawpularity: ' + str(average[6]))

print('Domestic Long Hair average Pawpularity: ' + str(average[52]))
print('Maltese average Pawpularity: ' + str(average[78]))

How about size? I guess this is mostly a dog-issue since cats don't differ _that_ much in size.

Class 35, Chihuahua and 102, Scotch Terrier against 62, Great Dane and 100, Saint Bernard...

Not much of a pattern in these 4 examples... further investigation could be interesting.

In [ ]:
print('Chihuahua average Pawpularity: ' + str(average[35]))
print('Scots Terrier average Pawpularity: ' + str(average[62]))

print('Great Dane average Pawpularity: ' + str(average[62]))
print('Saint Bernard average Pawpularity: ' + str(average[100]))


So there you have it. The classifications are not perfect (far from it) but I think I am unto something here and this should at least be a starting point for anyone agreeing that this is an attribute that should be taken into consideration.

Unfortunately for some of the pets it seems like they simply have their breed against them which really is not that much of a relevation after all... :-)
